In [1]:
# basic dependencies

import numpy as np
from numpy import loadtxt
from numpy import savetxt

import pandas as pd
import math
import time

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

###########

# torch dependencies
import torch

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")} # set tensors to GPU, if multiple GPUs please set cuda:x properly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_printoptions(precision=3)

###########

# botorch dependencies
import botorch

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP, FixedNoiseGP
from botorch.models.gp_regression_mixed import MixedSingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list, optimize_acqf_mixed
from botorch.acquisition.fixed_feature import FixedFeatureAcquisitionFunction
from botorch.acquisition.objective import GenericMCObjective, ConstrainedMCObjective
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement

# rest of the training loop
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume

# Sam's work on ref point inferrence & feasibility weighing
from botorch.utils.multi_objective.hypervolume import infer_reference_point
from botorch.acquisition.multi_objective.objective import MCMultiOutputObjective
from botorch.acquisition.utils import get_infeasible_cost
from typing import Optional
from torch import Tensor
from botorch.utils import apply_constraints

# argument for adding feasiility weighting to outcomes
class GenericMCMultiOutputObjective(GenericMCObjective, MCMultiOutputObjective):
    pass

# others
from botorch.exceptions import BadInitialCandidatesWarning
import warnings

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [2]:
!python --version
print('Numpy '+np.__version__)
print('Pandas '+pd.__version__)
print('PyTorch '+torch.__version__)
print('BoTorch '+botorch.__version__)

print('\nCuda device {}'.format(torch.cuda.get_device_name(torch.cuda.current_device())))

Python 3.8.8
Numpy 1.20.1
Pandas 1.2.4
PyTorch 1.10.0
BoTorch 0.5.1

Cuda device NVIDIA GeForce GTX 1650


In [3]:
df = pd.read_csv('multiobjdata.csv')
df

,S1 (g/L),S2 (g/L),S3 (g/L),P1 (g/L),T1 (g/L),Turbidity (NTU),Viscosity (mPas),Price ($/L)
0,0.4300,0.60,13.97,0.53,0.1800,27.75,6.96,0.103561
1,0.0870,0.57,14.34,0.42,0.0200,19.84,6.37,0.098841
2,0.6100,1.31,13.08,1.19,1.3600,292.37,573.87,0.119770
3,0.1500,0.50,14.35,0.16,0.0850,32.23,8.26,0.097497
4,0.0038,5.21,9.79,0.14,0.0087,43.61,8.51,0.096209
...,...,...,...,...,...,...,...,...
123,8.1300,3.36,3.51,0.37,0.8500,34.12,510.04,0.163156
124,1.8500,3.52,9.64,0.16,1.4200,19.62,528.88,0.120082
125,5.3700,0.56,9.07,0.68,1.9600,22.13,3449.00,0.153990
126,0.4800,6.82,7.70,0.33,1.7000,469.36,2427.21,0.114223


In [4]:
# initialization
random_state = 1
noise = 0
initial_size = 16
q = 8
verbose = True

X = df.iloc[:,0:5]
y1 = -df['Turbidity (NTU)']
y2 = -(df['Viscosity (mPas)'] - 3)**2
y = pd.concat([y1, y2], axis=1)

X_pool = torch.tensor(np.array(X), **tkwargs)
y_pool = torch.tensor(np.array(y), **tkwargs)

N_BATCH = (X_pool.shape[0]-initial_size)/q

assert N_BATCH%1 == 0.0, "No of initial samples, batch size and total pool size must tally"

In [5]:
# generate initial training data for that run
pareto_mask = is_non_dominated(y_pool) # check for 2nd criteria: non-dominated, meaning new pareto optimal
pareto_y = y_pool[pareto_mask]
ref_point = infer_reference_point(pareto_y)
ref_point = -ref_point
hvs = []
hv=Hypervolume(ref_point=-ref_point) # sets the hv based on problem, flip since BoTorch takes maximisation

torch.manual_seed(random_state)
perm = torch.randperm(X_pool.shape[0])
idx = perm[:16] # takes 16 samples
train_x = X_pool[idx]
train_obj = y_pool[idx]

x_mask = torch.ones(X_pool.shape[0], dtype=torch.bool)
x_mask[idx] = False
X_pool = X_pool[x_mask]

y_mask = torch.ones(y_pool.shape[0], dtype=torch.bool)
y_mask[idx] = False
y_pool = y_pool[y_mask]

models = []
for i in range(train_obj.shape[-1]):
    models.append(
        SingleTaskGP(train_x, train_obj[..., i:i+1], outcome_transform=Standardize(m=1))
    )
    
model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)

In [6]:
for iteration in range(1, int(N_BATCH)+1):
    
    t3 = time.time()

    # fit the surrogate model
    fit_gpytorch_model(mll)

    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=-ref_point, # for computing HV, must flip for BoTorch
        X_baseline=train_x, # feed total list of train_x for this current iteration
        objective=IdentityMCMultiOutputObjective(outcomes=np.arange(2).tolist()), # optimize first n_obj col 
        prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on

    acq_value_list = []

    for i in range(0, X_pool.shape[0]):
        with torch.no_grad():
            acq_value = acq_func(X_pool[i].unsqueeze(dim=0))
            acq_value_list.append(acq_value.item())

    #acq_value_list.sort(reverse=True)
    top_idx = sorted(range(len(acq_value_list)), key=lambda i: acq_value_list[i], reverse=True)[:q]
    new_x = X_pool[top_idx]
    new_obj = y_pool[top_idx]

    # update training points by concatenating the new values into their respective tensors
    train_x = torch.cat([train_x, new_x])
    train_obj = torch.cat([train_obj, new_obj])

    models = []
    for i in range(train_obj.shape[-1]):
        models.append(
            SingleTaskGP(train_x, train_obj[..., i:i+1], outcome_transform=Standardize(m=1))
        )

    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    
    x_mask = torch.ones(X_pool.shape[0], dtype=torch.bool)
    x_mask[top_idx] = False
    X_pool = X_pool[x_mask]

    y_mask = torch.ones(y_pool.shape[0], dtype=torch.bool)
    y_mask[top_idx] = False
    y_pool = y_pool[x_mask]
    
    # computing HV of current candidate list
    pareto_mask = is_non_dominated(train_obj) # check for 2nd criteria: non-dominated, meaning new pareto optimal
    pareto_y = train_obj[pareto_mask] # take only points that fit the 2nd check
    volume = hv.compute(pareto_y) # compute change in HV with new pareto optimal wrt to original ref point

    hvs.append(volume)
    
    t4 = time.time()
    if verbose:
        print(
                f"Batch {iteration:>2} of {int(N_BATCH)}: Hypervolume = "
                f"{hvs[-1]:>4.2f}, "
                f"time = {t4-t3:>4.2f}s, "
                f"remaining samples in pool: {y_pool.shape[0]}\n"
                , end="")
                    
print("DONE!")

Batch  1 of 14: Hypervolume = 7927.67, time = 2.00s, remaining samples in pool: 104
Batch  2 of 14: Hypervolume = 7927.67, time = 1.28s, remaining samples in pool: 96
Batch  3 of 14: Hypervolume = 8033.07, time = 1.25s, remaining samples in pool: 88
Batch  4 of 14: Hypervolume = 8033.07, time = 2.03s, remaining samples in pool: 80
Batch  5 of 14: Hypervolume = 8038.00, time = 2.13s, remaining samples in pool: 72
Batch  6 of 14: Hypervolume = 11424.14, time = 1.86s, remaining samples in pool: 64
Batch  7 of 14: Hypervolume = 13791.47, time = 2.01s, remaining samples in pool: 56
Batch  8 of 14: Hypervolume = 13791.47, time = 1.89s, remaining samples in pool: 48
Batch  9 of 14: Hypervolume = 20946.85, time = 2.11s, remaining samples in pool: 40
Batch 10 of 14: Hypervolume = 21088.15, time = 1.99s, remaining samples in pool: 32
Batch 11 of 14: Hypervolume = 21088.15, time = 3.54s, remaining samples in pool: 24
Batch 12 of 14: Hypervolume = 21088.15, time = 3.72s, remaining samples in pool: